In [2]:
import sys
sys.path.append('../src')

import pandas as pd
from data.notes_loader import NotesLoader
from data.text_preprocessing import TextPreprocessor, TextConfig
from data.text_integration import TextTabularIntegrator, create_stratified_splits


## ============================================================
## PASSO 1: CARREGAR DADOS TABULARES
## ============================================================

In [3]:
print("📊 PASSO 1: Carregando dados tabulares...")
edstays = pd.read_parquet('../data/processed/labeled_data.parquet')

print(f"✓ ED stays carregados: {len(edstays):,}")
print(f"  - Pacientes: {edstays['subject_id'].nunique():,}")
print(f"  - Critical Outcome: {edstays['critical_outcome'].sum():,} ({edstays['critical_outcome'].mean()*100:.1f}%)")
print(f"  - Lengthened ED Stay: {edstays['lengthened_ed_stay'].sum():,} ({edstays['lengthened_ed_stay'].mean()*100:.1f}%)")

📊 PASSO 1: Carregando dados tabulares...
✓ ED stays carregados: 189,158
  - Pacientes: 99,346
  - Critical Outcome: 10,105 (5.3%)
  - Lengthened ED Stay: 11,227 (5.9%)


## ============================================================
## PASSO 2: CARREGAR E FILTRAR NOTAS
## ============================================================

In [4]:
print("\n📖 PASSO 2: Carregando notas clínicas...")
loader = NotesLoader(data_root='../data/raw/')

# Carregar notas brutas
notes = loader.load_notes(
    categories=['discharge', 'radiology'],
    subject_ids=edstays['subject_id'].unique().tolist(),
    hadm_ids=edstays['hadm_id'].unique().tolist(),
    chunksize=10000
)

print(f"✓ Notas carregadas: {len(notes):,}")

# Aplicar estratégia híbrida
print("\n🎯 Aplicando estratégia híbrida...")

rad_notes = notes[notes['note_category'] == 'radiology'].copy()
dis_notes = notes[notes['note_category'] == 'discharge'].copy()

rad_filtered = loader.filter_temporal(rad_notes, edstays, time_buffer_hours=0)
dis_filtered = loader.filter_temporal(dis_notes, edstays, time_buffer_hours=12)

notes_temporal = pd.concat([rad_filtered, dis_filtered], ignore_index=True)

print(f"\n✓ Notas após filtro híbrido: {len(notes_temporal):,}")
print(f"  - Radiology (0h):  {len(rad_filtered):,}")
print(f"  - Discharge (12h): {len(dis_filtered):,}")


INFO:data.notes_loader:✓ Categorias disponíveis: discharge, radiology
INFO:data.notes_loader:📖 Carregando discharge notes em chunks de 10,000...
INFO:data.notes_loader:   Carregando colunas: ['subject_id', 'hadm_id', 'charttime', 'storetime', 'text', 'note_id', 'note_type', 'note_seq']
INFO:data.notes_loader:   Parseando datas: ['charttime', 'storetime']



📖 PASSO 2: Carregando notas clínicas...


INFO:data.notes_loader:   Processados 10 chunks, 21,744 notas mantidas
INFO:data.notes_loader:   Processados 20 chunks, 43,674 notas mantidas
INFO:data.notes_loader:   Processados 30 chunks, 65,552 notas mantidas
INFO:data.notes_loader:✓ discharge: 72,544 notas carregadas
INFO:data.notes_loader:📖 Carregando radiology notes em chunks de 10,000...
INFO:data.notes_loader:   Carregando colunas: ['subject_id', 'hadm_id', 'charttime', 'storetime', 'text', 'note_id', 'note_type', 'note_seq']
INFO:data.notes_loader:   Parseando datas: ['charttime', 'storetime']
INFO:data.notes_loader:   Processados 10 chunks, 10,302 notas mantidas
INFO:data.notes_loader:   Processados 20 chunks, 20,758 notas mantidas
INFO:data.notes_loader:   Processados 30 chunks, 31,974 notas mantidas
INFO:data.notes_loader:   Processados 40 chunks, 41,738 notas mantidas
INFO:data.notes_loader:   Processados 50 chunks, 52,557 notas mantidas
INFO:data.notes_loader:   Processados 60 chunks, 63,327 notas mantidas
INFO:data.note

✓ Notas carregadas: 315,202

🎯 Aplicando estratégia híbrida...


INFO:data.notes_loader:  - Notas após merge com ED stays: 240,693
INFO:data.notes_loader:  - Notas com timestamps válidos: 240,693
INFO:data.notes_loader:✓ Filtro temporal: 242,658 → 95,275 notas (39.3% mantidas)
INFO:data.notes_loader:⏱️ Aplicando filtro temporal nas notas...
INFO:data.notes_loader:  - Notas com hadm_id válido: 72,544
INFO:data.notes_loader:  - Notas após merge com ED stays: 71,862
INFO:data.notes_loader:  - Notas com timestamps válidos: 71,862
INFO:data.notes_loader:✓ Filtro temporal: 72,544 → 9,690 notas (13.4% mantidas)



✓ Notas após filtro híbrido: 104,965
  - Radiology (0h):  95,275
  - Discharge (12h): 9,690


## ============================================================
## PASSO 3: PRÉ-PROCESSAMENTO DE TEXTO
## ============================================================

In [5]:
print("\n🧹 PASSO 3: Limpando texto...")

config = TextConfig(
    max_tokens_per_segment=512,
    min_text_length=50,
    lowercase=False  # Manter case para nomes de drogas
)

preprocessor = TextPreprocessor(config)
notes_clean = preprocessor.preprocess_dataframe(notes_temporal, text_column='text')

# Filtrar apenas notas válidas
notes_valid = notes_clean[notes_clean['is_valid']].copy()

print(f"✓ Notas válidas após limpeza: {len(notes_valid):,}")
print(f"  - Taxa de validação: {len(notes_valid)/len(notes_temporal)*100:.1f}%")

# Estatísticas de texto
print(f"\n📊 Estatísticas de texto limpo:")
print(f"  - Tamanho médio: {notes_valid['cleaned_length'].mean():.0f} chars")
print(f"  - Mediana: {notes_valid['cleaned_length'].median():.0f} chars")

INFO:data.text_preprocessing:🧹 Pré-processando 104,965 notas...



🧹 PASSO 3: Limpando texto...


INFO:data.text_preprocessing:✓ Notas válidas: 104,952/104,965


✓ Notas válidas após limpeza: 104,952
  - Taxa de validação: 100.0%

📊 Estatísticas de texto limpo:
  - Tamanho médio: 1904 chars
  - Mediana: 911 chars


## ============================================================
## PASSO 4: INTEGRAÇÃO TEXTO-TABULAR
## ============================================================

In [6]:
print("\n🔗 PASSO 4: Integrando texto com dados tabulares...")

integrator = TextTabularIntegrator()

# Associar notas aos ED stays (priorizar discharge > radiology)
df_integrated = integrator.associate_notes_to_stays(
    df_tabular=edstays,
    notes=notes_valid,
    strategy='priority'  # discharge tem prioridade sobre radiology
)

# Estatísticas de integração
stats = integrator.get_statistics(df_integrated)
print(f"\n✓ Integração completa:")
print(f"  - Total ED stays: {stats['total_stays']:,}")
print(f"  - Com texto: {stats['with_text']:,}")
print(f"  - Cobertura: {stats['text_coverage']*100:.1f}%")
print(f"  - Tamanho médio: {stats['avg_text_length']:.0f} chars")

# Distribuição por categoria
if 'by_category' in stats:
    print(f"\n  Distribuição por tipo de nota:")
    for cat, count in stats['by_category'].items():
        print(f"    - {cat}: {count:,}")

INFO:data.text_integration:🔗 Associando notas aos stays (strategy=priority)...



🔗 PASSO 4: Integrando texto com dados tabulares...


INFO:data.text_integration:✓ 61,924 stays com texto



✓ Integração completa:
  - Total ED stays: 189,158
  - Com texto: 61,924
  - Cobertura: 32.7%
  - Tamanho médio: 2182 chars

  Distribuição por tipo de nota:
    - RR: 54,207
    - DS: 7,653
    - AR: 64


## ============================================================
## PASSO 5: CRIAR DATASET MULTI-MODAL
## ============================================================

In [7]:
print("\n🎨 PASSO 5: Criando dataset multi-modal...")

dataset = integrator.create_multimodal_dataset(
    df_integrated,
    text_column='cleaned_text',
    outcome_column='critical_outcome'  # Podemos criar datasets para ambos os outcomes
)

print(f"✓ Dataset criado: {dataset.shape}")
print(f"  - Features tabulares: {len([c for c in dataset.columns if c.startswith('triage_') or c.startswith('lab_')])}")
print(f"  - Com texto: {dataset['has_text'].sum():,} ({dataset['has_text'].mean()*100:.1f}%)")
print(f"  - Positivos: {dataset['outcome'].sum():,} ({dataset['outcome'].mean()*100:.1f}%)")


INFO:data.text_integration:🎨 Criando dataset multi-modal...
INFO:data.text_integration:✓ Dataset criado: (189158, 57)



🎨 PASSO 5: Criando dataset multi-modal...
✓ Dataset criado: (189158, 57)
  - Features tabulares: 51
  - Com texto: 61,924 (32.7%)
  - Positivos: 10,105 (5.3%)


## ============================================================
## PASSO 6: SPLITS ESTRATIFICADOS
## ============================================================

In [8]:
print("\n✂️ PASSO 6: Criando splits estratificados...")

splits = create_stratified_splits(
    dataset,
    outcome_col='outcome',
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42
)

# Salvar splits
for split_name, split_df in splits.items():
    output_path = f'../data/processed/multimodal_{split_name}.parquet'
    split_df.to_parquet(output_path, index=False)
    
    # Estatísticas do split
    n_with_text = split_df['has_text'].sum()
    n_positive = split_df['outcome'].sum()
    
    print(f"\n✓ {split_name.upper():5s}: {len(split_df):5,} amostras")
    print(f"     - Com texto: {n_with_text:5,} ({n_with_text/len(split_df)*100:4.1f}%)")
    print(f"     - Positivos: {n_positive:5,} ({n_positive/len(split_df)*100:4.1f}%)")


✂️ PASSO 6: Criando splits estratificados...


INFO:data.text_integration:✓ Splits criados: train=151326, val=18916, test=18916



✓ TRAIN: 151,326 amostras
     - Com texto: 49,599 (32.8%)
     - Positivos: 8,084 ( 5.3%)

✓ VAL  : 18,916 amostras
     - Com texto: 6,146 (32.5%)
     - Positivos: 1,011 ( 5.3%)

✓ TEST : 18,916 amostras
     - Com texto: 6,179 (32.7%)
     - Positivos: 1,010 ( 5.3%)


## ============================================================
## PASSO 7: VALIDAÇÃO FINAL
## ============================================================

In [9]:

print("\n\n" + "="*60)
print("✅ PIPELINE DE DADOS TEXTUAIS CONCLUÍDO")
print("="*60)

print("\n📁 Arquivos gerados:")
print("  ✓ multimodal_train.parquet")
print("  ✓ multimodal_val.parquet")
print("  ✓ multimodal_test.parquet")

print("\n🎯 Próximos passos:")
print("  1. Implementar linearização de features (Fase 3)")
print("  2. Fine-tune BioGPT (SFT)")
print("  3. Treinamento com RL (PPO)")
print("  4. Comparação: Tabular vs. Multi-modal")

print("\n" + "="*60)



✅ PIPELINE DE DADOS TEXTUAIS CONCLUÍDO

📁 Arquivos gerados:
  ✓ multimodal_train.parquet
  ✓ multimodal_val.parquet
  ✓ multimodal_test.parquet

🎯 Próximos passos:
  1. Implementar linearização de features (Fase 3)
  2. Fine-tune BioGPT (SFT)
  3. Treinamento com RL (PPO)
  4. Comparação: Tabular vs. Multi-modal



In [10]:
# ============================================================
# VALIDAÇÃO DOS SPLITS
# ============================================================
import pandas as pd

print("🔍 VALIDAÇÃO DOS SPLITS\n")
print("="*60)

for split_name in ['train', 'val', 'test']:
    df = pd.read_parquet(f'../data/processed/multimodal_{split_name}.parquet')
    
    print(f"\n{split_name.upper()}:")
    print(f"  - Shape: {df.shape}")
    print(f"  - Colunas: {list(df.columns)[:10]}... ({len(df.columns)} total)")
    
    # Verificar integridade
    print(f"\n  ✓ Integridade:")
    print(f"    - Nulls em 'outcome': {df['outcome'].isnull().sum()}")
    print(f"    - Nulls em 'has_text': {df['has_text'].isnull().sum()}")
    print(f"    - Texto vazio (quando has_text=True): {(df['has_text'] & df['cleaned_text'].isnull()).sum()}")
    
    # Distribuição
    print(f"\n  📊 Distribuição:")
    print(f"    - Com texto: {df['has_text'].mean()*100:.1f}%")
    print(f"    - Outcome positivo: {df['outcome'].mean()*100:.1f}%")
    
    # Estratificação
    print(f"\n  🎯 Estratificação (texto x outcome):")
    crosstab = pd.crosstab(df['has_text'], df['outcome'], normalize='columns') * 100
    print(crosstab.round(1))

print("\n" + "="*60)
print("✅ VALIDAÇÃO CONCLUÍDA")
print("="*60)

🔍 VALIDAÇÃO DOS SPLITS


TRAIN:
  - Shape: (151326, 57)
  - Colunas: ['subject_id', 'hadm_id', 'stay_id', 'cleaned_text', 'outcome', 'triage_completeness', 'triage_age', 'triage_gender_male', 'triage_gender_female', 'triage_heart_rate']... (57 total)

  ✓ Integridade:
    - Nulls em 'outcome': 0
    - Nulls em 'has_text': 0
    - Texto vazio (quando has_text=True): 0

  📊 Distribuição:
    - Com texto: 32.8%
    - Outcome positivo: 5.3%

  🎯 Estratificação (texto x outcome):
outcome      0     1
has_text            
False     69.5  26.9
True      30.5  73.1

VAL:
  - Shape: (18916, 57)
  - Colunas: ['subject_id', 'hadm_id', 'stay_id', 'cleaned_text', 'outcome', 'triage_completeness', 'triage_age', 'triage_gender_male', 'triage_gender_female', 'triage_heart_rate']... (57 total)

  ✓ Integridade:
    - Nulls em 'outcome': 0
    - Nulls em 'has_text': 0
    - Texto vazio (quando has_text=True): 0

  📊 Distribuição:
    - Com texto: 32.5%
    - Outcome positivo: 5.3%

  🎯 Estratificação (t